In [152]:
!pip install pandas numpy

In [153]:
import pandas as pd
import numpy as np

In [154]:
df = pd.read_csv('data.csv')
df.head()

,name,facility,price,construction_year,distance,security
0,Apartement 1,2,7000000,2012,7,3
1,Apartement 2,4,10000000,2015,2,3
2,Apartement 3,3,8500000,2010,4,4


In [155]:
# c1: Fasilitas pendukung di apartemen (bobot: 30)
# c2: Harga bangunan per meter persegi (bobot: 20)
# c3: Tahun konstruksi bangunan apartemen (bobot: 20)
# c4: Jarak dari tempat kerja (dalam kilometer) (bobot: 20)
# c5: Sistem keamanan apartemen (bobot: 10) 

# Convert from raw 'price' to match the criteria

# Define a custom function to apply the criteria
def transform_price(price):
    if price >= 10000000:
        return 1
    elif 5000000 <= price < 10000000:
        return 2
    elif 1000000 <= price < 5000000:
        return 3
    else:
        return None  # Handle cases where price doesn't match any criteria

In [156]:
df['price'] = df['price'].apply(transform_price)
df.head()

,name,facility,price,construction_year,distance,security
0,Apartement 1,2,2,2012,7,3
1,Apartement 2,4,1,2015,2,3
2,Apartement 3,3,2,2010,4,4


In [157]:
def transform_year(year):
    if year >= 2015:
        return 3
    elif 2010 <= year < 2015:
        return 2
    elif 2005 <= year < 2010:
        return 1
    else:
        return None  # Handle cases where year doesn't match any criteria

In [158]:
df['construction_year'] = df['construction_year'].apply(transform_year)
df.head()

,name,facility,price,construction_year,distance,security
0,Apartement 1,2,2,2,7,3
1,Apartement 2,4,1,3,2,3
2,Apartement 3,3,2,2,4,4


In [159]:
def normalize_benefit(value, max):
    return value / max

def normalize_cost(value, min):
    return min / value

In [160]:
max_facility = df['facility'].max()
min_price = df['price'].min()
max_construction_year = df['construction_year'].max()
min_distance = df['distance'].min()
max_security = df['security'].max()

[max_facility, min_price, max_construction_year, min_distance, max_security]

[4, 1, 3, 2, 4]

In [161]:
# normalize

# mark, distance and price as cost
# and the rest is benefit

df['facility'] = df.apply(lambda x: normalize_benefit(x['facility'], float(max_facility)),axis=1)
df['price'] = df.apply(lambda x: normalize_cost(x['price'], float(min_price)),axis=1)
df['construction_year'] = df.apply(lambda x: normalize_benefit(x['construction_year'], float(max_construction_year)),axis=1)
df['distance'] = df.apply(lambda x: normalize_cost(x['distance'], float(min_distance)),axis=1)
df['security'] = df.apply(lambda x: normalize_benefit(x['security'], float(max_security)),axis=1)

df.head()

,name,facility,price,construction_year,distance,security
0,Apartement 1,0.50,0.5,0.666667,0.285714,0.75
1,Apartement 2,1.00,1.0,1.000000,1.000000,0.75
2,Apartement 3,0.75,0.5,0.666667,0.500000,1.00


In [162]:
# weight

weight = {
    'facility': 0.3,
    'price': 0.2,
    'construction_year': 0.2,
    'distance': 0.2,
    'security': 0.1
}

In [163]:
def apply_weight_wsm(dataframe, weight_dict):
    for column, weight in weight_dict.items():
        if column in dataframe.columns:
            dataframe[column] *= weight
    return dataframe

In [164]:
df = apply_weight_wsm(df.copy(), weight)

df_copy = df.copy()

df.head()

,name,facility,price,construction_year,distance,security
0,Apartement 1,0.150,0.1,0.133333,0.057143,0.075
1,Apartement 2,0.300,0.2,0.200000,0.200000,0.075
2,Apartement 3,0.225,0.1,0.133333,0.100000,0.100


In [165]:
df['wsm_score'] = df[['facility', 'price', 'construction_year', 'distance', 'security']].sum(axis=1)
df.head()

,name,facility,price,construction_year,distance,security,wsm_score
0,Apartement 1,0.150,0.1,0.133333,0.057143,0.075,0.515476
1,Apartement 2,0.300,0.2,0.200000,0.200000,0.075,0.975000
2,Apartement 3,0.225,0.1,0.133333,0.100000,0.100,0.658333


In [166]:
# Answer -> WSM Result
df = df.sort_values(by='wsm_score', ascending=False)
df.head()


,name,facility,price,construction_year,distance,security,wsm_score
1,Apartement 2,0.300,0.2,0.200000,0.200000,0.075,0.975000
2,Apartement 3,0.225,0.1,0.133333,0.100000,0.100,0.658333
0,Apartement 1,0.150,0.1,0.133333,0.057143,0.075,0.515476


In [167]:
def calculate_wpm_score(row):
    normalized_values = row[1:]  # Exclude the 'name' column
    wpm_scores = [value ** weight for value, weight in zip(normalized_values, weight.values())]
    return pd.Series(wpm_scores)

In [168]:
# Answer -> WPM Result

wpm_scores_df = df.apply(calculate_wpm_score, axis=1)

# Calculate the product of WPM scores for each alternative
df['wpm_score'] = wpm_scores_df.prod(axis=1)
df = df.sort_values(by='wpm_score', ascending=False)
df.head()

,name,facility,price,construction_year,distance,security,wsm_score,wpm_score
1,Apartement 2,0.300,0.2,0.200000,0.200000,0.075,0.975000,0.204767
2,Apartement 3,0.225,0.1,0.133333,0.100000,0.100,0.658333,0.135096
0,Apartement 1,0.150,0.1,0.133333,0.057143,0.075,0.515476,0.103923
